# 04 — Real Neural Data (LFP)

This notebook applies TVAR models to real neural recordings.

## Datasets
- **Mice LFP**: Local field potentials from mouse recordings
- **Monkey LFP**: Primate neural recordings

## Pipeline
1. Data loading & preprocessing
2. Spectral characterization
3. TVAR model fitting
4. Interpretation of time-varying dynamics

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch, butter, filtfilt, spectrogram, hilbert, iirnotch, sosfiltfilt, decimate
from scipy.ndimage import gaussian_filter, gaussian_filter1d

---
# Part 1: Mouse LFP Pipeline (Hippocampal Recordings)

**Focus**: Sharp-wave ripple (SWR) detection in hippocampal LFP
- **Sampling rate**: ~20 kHz (typical for high-density probes)
- **Key features**: 1/f background, ripple oscillations (80-140 Hz)
- **Pipeline steps**:
  1. Load raw LFP data
  2. Bandpass filter for ripple band (80-140 Hz)
  3. Compute Hilbert envelope
  4. Threshold-based ripple detection
  5. Spectrogram visualization

In [ ]:
# ============================================================
# Mouse LFP Processing Functions
# ============================================================

def bandpass_filter(data: np.ndarray, lowcut: float, highcut: float, 
                    fs: float, order: int = 4) -> np.ndarray:
    """
    Apply a Butterworth bandpass filter.
    
    Parameters
    ----------
    data : np.ndarray
        Input signal
    lowcut : float
        Low cutoff frequency (Hz)
    highcut : float
        High cutoff frequency (Hz)
    fs : float
        Sampling frequency (Hz)
    order : int
        Filter order
        
    Returns
    -------
    np.ndarray
        Filtered signal
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)


def detect_ripples(envelope: np.ndarray, time: np.ndarray, 
                   threshold_sd: float = 3.0) -> list:
    """
    Detect ripple intervals based on envelope threshold crossing.
    
    Parameters
    ----------
    envelope : np.ndarray
        Hilbert envelope of the ripple-band signal
    time : np.ndarray
        Time vector
    threshold_sd : float
        Number of standard deviations for threshold
        
    Returns
    -------
    list
        List of (start_time, end_time) tuples for each ripple
    """
    threshold = threshold_sd * np.std(envelope)
    ripple_intervals = []
    in_ripple = False
    
    for i, val in enumerate(envelope):
        if val > threshold and not in_ripple:
            start = time[i]
            in_ripple = True
        elif val <= threshold and in_ripple:
            end = time[i]
            ripple_intervals.append((start, end))
            in_ripple = False
            
    return ripple_intervals, threshold


def simulate_mouse_lfp(fs: float, duration: float, ripple_freq: float = 120,
                       ripple_duration: float = 0.1, ripple_amplitude: float = 50,
                       ripple_count: int = 5, seed: int = None) -> tuple:
    """
    Simulate a realistic mouse hippocampal LFP with 1/f power decay and ripples.
    
    Parameters
    ----------
    fs : float
        Sampling frequency (Hz)
    duration : float
        Signal duration (seconds)
    ripple_freq : float
        Ripple oscillation frequency (Hz)
    ripple_duration : float
        Duration of each ripple event (seconds)
    ripple_amplitude : float
        Amplitude of ripples (µV)
    ripple_count : int
        Number of ripple events to embed
    seed : int
        Random seed for reproducibility
        
    Returns
    -------
    tuple
        (time, lfp) arrays
    """
    if seed is not None:
        np.random.seed(seed)
    
    time = np.linspace(0, duration, int(duration * fs))
    
    # Generate 1/f^2 noise (Brown noise)
    freqs = np.fft.rfftfreq(len(time), d=1/fs)
    amplitudes = np.zeros_like(freqs)
    amplitudes[1:] = 1 / (freqs[1:] ** 2)
    phases = np.random.uniform(0, 2 * np.pi, len(amplitudes))
    brown_noise = np.fft.irfft(amplitudes * np.exp(1j * phases))
    brown_noise = brown_noise[:len(time)]
    brown_noise = brown_noise / np.std(brown_noise) * 200
    
    # Add multiple frequency components with 1/f amplitude decay
    freq_bands = [1, 3, 4, 7, 10, 20, 30, 35, 40, 80, 110]
    freq_components = np.zeros_like(time)
    for freq in freq_bands:
        amplitude = 150 / np.sqrt(freq)
        freq_components += amplitude * np.sin(2 * np.pi * freq * time + np.random.uniform(0, 2 * np.pi))
    
    lfp = brown_noise + freq_components
    
    # Add synthetic ripples at random times
    ripple_times = np.random.uniform(2, duration - 2, ripple_count)
    for ripple_time in ripple_times:
        start = int((ripple_time - ripple_duration / 2) * fs)
        end = int((ripple_time + ripple_duration / 2) * fs)
        lfp[start:end] += ripple_amplitude * np.sin(2 * np.pi * ripple_freq * time[start:end])
    
    return time, lfp


print("Mouse LFP processing functions loaded.")

In [ ]:
# ============================================================
# Mouse LFP Pipeline: Demonstration with Simulated Data
# ============================================================

# Generate simulated mouse hippocampal LFP
fs_mouse = 1000  # Hz
duration = 10    # seconds

time_mouse, raw_lfp_mouse = simulate_mouse_lfp(
    fs=fs_mouse, duration=duration, 
    ripple_freq=120, ripple_count=5, seed=42
)

# Step 1: Bandpass filter for ripple band (80-140 Hz)
ripple_band_lfp = bandpass_filter(raw_lfp_mouse, 80, 140, fs_mouse)

# Step 2: Compute envelope using Hilbert transform
analytic_signal = hilbert(ripple_band_lfp)
envelope = np.abs(analytic_signal)

# Step 3: Detect ripples based on threshold
ripple_intervals, ripple_threshold = detect_ripples(envelope, time_mouse, threshold_sd=3.0)

# Step 4: Compute spectrogram
f_spec, t_spec, Sxx = spectrogram(raw_lfp_mouse, fs_mouse, nperseg=256, noverlap=200)
Sxx_smooth = gaussian_filter(Sxx, sigma=3)

print(f"Detected {len(ripple_intervals)} ripple events")

In [ ]:
# ============================================================
# Mouse LFP Visualization
# ============================================================

fig, axs = plt.subplots(5, 1, figsize=(12, 12), sharex=True)

# Raw LFP
axs[0].plot(time_mouse, raw_lfp_mouse, color='black', lw=0.5)
axs[0].set_title("Raw LFP (Mouse Hippocampus)")
axs[0].set_ylabel("Amplitude (µV)")

# Ripple band LFP
axs[1].plot(time_mouse, ripple_band_lfp, color='gray', lw=0.5)
axs[1].set_title("Ripple Band LFP (80-140 Hz)")
axs[1].set_ylabel("Amplitude (µV)")

# Ripple band envelope
axs[2].plot(time_mouse, envelope, color='black', lw=0.8)
axs[2].axhline(ripple_threshold, color='red', linestyle='--', label="Threshold (3 SD)")
axs[2].set_title("Ripple Band Envelope (Hilbert)")
axs[2].set_ylabel("Amplitude (µV)")
axs[2].legend()

# Smoothed spectrogram
im = axs[3].pcolormesh(t_spec, f_spec, 10 * np.log10(Sxx_smooth + 1e-12), 
                        shading='gouraud', cmap='viridis')
axs[3].set_title("Spectrogram (Smoothed)")
axs[3].set_ylabel("Frequency (Hz)")
cbar_ax = axs[3].inset_axes([0.85, 0.1, 0.02, 0.8])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='vertical')
cbar.set_label('Power (dB)')

# Raw LFP with ripple intervals
axs[4].plot(time_mouse, raw_lfp_mouse, color='black', lw=0.5)
for i, interval in enumerate(ripple_intervals):
    label = "Ripple Interval" if i == 0 else ""
    axs[4].axvspan(interval[0], interval[1], color='green', alpha=0.5, label=label)
axs[4].set_title("Raw LFP with Detected Ripples")
axs[4].set_ylabel("Amplitude (µV)")
axs[4].set_xlabel("Time (s)")
if ripple_intervals:
    axs[4].legend()

for ax in axs:
    ax.set_xlim([0, duration])

plt.tight_layout()
plt.show()

---
# Part 2: Monkey LFP Pipeline (Visual Cortex Recordings)

**Focus**: Multi-array V1/V4 LFP analysis for cross-area dynamics
- **Sampling rate**: 500 Hz (typical for Utah arrays)
- **Key features**: Visual cortex oscillations, cross-area coherence
- **Pipeline steps**:
  1. Load multi-channel LFP data (multiple arrays)
  2. Notch filter (60/120 Hz line noise)
  3. Bandpass filter (0.5-200 Hz)
  4. Channel grouping by brain region (V1 vs V4)
  5. PSD analysis and comparison
  6. Optional downsampling for AR modeling

In [ ]:
# ============================================================
# Monkey LFP Processing Functions
# ============================================================

def notch_filter(data: np.ndarray, fs: float, 
                 freqs: tuple = (60.0, 120.0), Q: float = 30.0) -> np.ndarray:
    """
    Apply notch filters to remove line noise.
    
    Parameters
    ----------
    data : np.ndarray
        Input signal
    fs : float
        Sampling frequency (Hz)
    freqs : tuple
        Frequencies to notch out (Hz)
    Q : float
        Quality factor
        
    Returns
    -------
    np.ndarray
        Filtered signal
    """
    y = data.astype(float)
    for f0 in freqs:
        if f0 < fs / 2:
            b, a = iirnotch(f0, Q, fs)
            y = filtfilt(b, a, y)
    return y


def butter_bandpass(data: np.ndarray, fs: float, 
                    hp: float = None, lp: float = None, 
                    order: int = 4) -> np.ndarray:
    """
    Apply Butterworth high-pass, low-pass, or band-pass filter.
    
    Parameters
    ----------
    data : np.ndarray
        Input signal
    fs : float
        Sampling frequency (Hz)
    hp : float
        High-pass cutoff (Hz), or None
    lp : float
        Low-pass cutoff (Hz), or None
    order : int
        Filter order
        
    Returns
    -------
    np.ndarray
        Filtered signal
    """
    nyquist = fs / 2
    if hp and lp:
        b, a = butter(order, [hp / nyquist, lp / nyquist], btype='band')
    elif hp:
        b, a = butter(order, hp / nyquist, btype='high')
    elif lp:
        b, a = butter(order, lp / nyquist, btype='low')
    else:
        return data
    return filtfilt(b, a, data)


def preprocess_monkey_lfp(raw_lfp: np.ndarray, fs: float, 
                          hp: float = 0.5, lp: float = 200.0,
                          notch_freqs: tuple = (60.0, 120.0),
                          target_fs: float = None) -> tuple:
    """
    Full preprocessing pipeline for monkey visual cortex LFP.
    
    Parameters
    ----------
    raw_lfp : np.ndarray
        Raw LFP signal (1D or 2D: time x channels)
    fs : float
        Original sampling frequency (Hz)
    hp : float
        High-pass cutoff (Hz)
    lp : float
        Low-pass cutoff (Hz)
    notch_freqs : tuple
        Line noise frequencies to remove
    target_fs : float
        Target sampling rate for downsampling (None to skip)
        
    Returns
    -------
    tuple
        (processed_lfp, new_fs)
    """
    # Handle 1D vs 2D input
    if raw_lfp.ndim == 1:
        y = raw_lfp.astype(float)
    else:
        y = raw_lfp.astype(float)
    
    # Notch filter for line noise
    if y.ndim == 1:
        y = notch_filter(y, fs, notch_freqs)
        y = butter_bandpass(y, fs, hp=hp, lp=lp)
    else:
        for ch in range(y.shape[1]):
            y[:, ch] = notch_filter(y[:, ch], fs, notch_freqs)
            y[:, ch] = butter_bandpass(y[:, ch], fs, hp=hp, lp=lp)
    
    # Optional downsampling
    new_fs = fs
    if target_fs and fs > target_fs:
        r = int(np.floor(fs / target_fs))
        if y.ndim == 1:
            y = decimate(y, r, ftype='iir', zero_phase=True)
        else:
            y = decimate(y, r, axis=0, ftype='iir', zero_phase=True)
        new_fs = fs / r
    
    return y, new_fs


def compute_psd_welch(data: np.ndarray, fs: float, 
                      fmin: float = 0.5, fmax: float = 200.0,
                      n_fft: int = 4096) -> tuple:
    """
    Compute power spectral density using Welch's method.
    
    Parameters
    ----------
    data : np.ndarray
        Input signal (1D)
    fs : float
        Sampling frequency (Hz)
    fmin : float
        Minimum frequency to return
    fmax : float
        Maximum frequency to return
    n_fft : int
        FFT window size
        
    Returns
    -------
    tuple
        (frequencies, psd)
    """
    freqs, psd = welch(data, fs=fs, nperseg=n_fft, noverlap=n_fft//2)
    mask = (freqs >= fmin) & (freqs <= fmax)
    return freqs[mask], psd[mask]


print("Monkey LFP processing functions loaded.")

In [ ]:
# ============================================================
# Monkey LFP Pipeline: Demonstration with Simulated Data
# ============================================================

def simulate_monkey_lfp(fs: float, duration: float, n_channels: int = 16,
                        seed: int = None) -> tuple:
    """
    Simulate multi-channel visual cortex LFP with realistic spectral properties.
    
    Parameters
    ----------
    fs : float
        Sampling frequency (Hz)
    duration : float
        Signal duration (seconds)
    n_channels : int
        Number of channels
    seed : int
        Random seed
        
    Returns
    -------
    tuple
        (time, lfp_array, channel_regions)
    """
    if seed is not None:
        np.random.seed(seed)
    
    n_samples = int(duration * fs)
    time = np.linspace(0, duration, n_samples)
    lfp = np.zeros((n_samples, n_channels))
    
    # Assign half to V1, half to V4
    regions = np.array(['V1'] * (n_channels // 2) + ['V4'] * (n_channels - n_channels // 2))
    
    for ch in range(n_channels):
        # 1/f background
        freqs = np.fft.rfftfreq(n_samples, d=1/fs)
        amplitudes = np.zeros_like(freqs)
        amplitudes[1:] = 1 / (freqs[1:] ** 1.2)  # 1/f^1.2 for LFP
        phases = np.random.uniform(0, 2 * np.pi, len(amplitudes))
        pink_noise = np.fft.irfft(amplitudes * np.exp(1j * phases))[:n_samples]
        pink_noise = pink_noise / np.std(pink_noise) * 50
        
        # Add oscillatory components (different for V1 vs V4)
        if regions[ch] == 'V1':
            # V1: stronger gamma (30-80 Hz)
            gamma = 20 * np.sin(2 * np.pi * 45 * time + np.random.uniform(0, 2*np.pi))
            alpha = 10 * np.sin(2 * np.pi * 10 * time + np.random.uniform(0, 2*np.pi))
            lfp[:, ch] = pink_noise + gamma + alpha
        else:
            # V4: stronger alpha/beta (8-30 Hz)
            beta = 15 * np.sin(2 * np.pi * 20 * time + np.random.uniform(0, 2*np.pi))
            alpha = 25 * np.sin(2 * np.pi * 10 * time + np.random.uniform(0, 2*np.pi))
            lfp[:, ch] = pink_noise + beta + alpha
        
        # Add 60 Hz line noise
        lfp[:, ch] += 5 * np.sin(2 * np.pi * 60 * time)
    
    return time, lfp, regions


# Generate simulated monkey visual cortex LFP
fs_monkey = 500   # Hz
duration_monkey = 20  # seconds
n_channels = 16

time_monkey, raw_lfp_monkey, channel_regions = simulate_monkey_lfp(
    fs=fs_monkey, duration=duration_monkey, n_channels=n_channels, seed=42
)

# Preprocess
processed_lfp_monkey, fs_processed = preprocess_monkey_lfp(
    raw_lfp_monkey, fs_monkey,
    hp=0.5, lp=200.0,
    notch_freqs=(60.0, 120.0),
    target_fs=None  # No downsampling needed, already at 500 Hz
)

# Separate V1 and V4 channels
v1_idx = np.where(channel_regions == 'V1')[0]
v4_idx = np.where(channel_regions == 'V4')[0]

print(f"Simulated {n_channels} channels: {len(v1_idx)} V1, {len(v4_idx)} V4")
print(f"Sampling rate: {fs_processed} Hz")

In [ ]:
# ============================================================
# Monkey LFP Visualization: PSD Comparison (V1 vs V4)
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Compute and plot PSDs for V1 channels
ax = axes[0]
for ch in v1_idx:
    freqs, psd = compute_psd_welch(processed_lfp_monkey[:, ch], fs_processed)
    psd_smooth = gaussian_filter1d(psd, sigma=2)
    ax.loglog(freqs, psd_smooth, color='navy', alpha=0.3, lw=0.8)

# Average V1 PSD
v1_psds = []
for ch in v1_idx:
    freqs, psd = compute_psd_welch(processed_lfp_monkey[:, ch], fs_processed)
    v1_psds.append(psd)
v1_avg = np.mean(v1_psds, axis=0)
v1_avg_smooth = gaussian_filter1d(v1_avg, sigma=2)
ax.loglog(freqs, v1_avg_smooth, color='navy', lw=2.5, label='V1 Average')
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("PSD (a.u./Hz)")
ax.set_title("V1 LFP Power Spectrum")
ax.legend()
ax.grid(True, ls='--', alpha=0.3)

# Compute and plot PSDs for V4 channels
ax = axes[1]
for ch in v4_idx:
    freqs, psd = compute_psd_welch(processed_lfp_monkey[:, ch], fs_processed)
    psd_smooth = gaussian_filter1d(psd, sigma=2)
    ax.loglog(freqs, psd_smooth, color='darkred', alpha=0.3, lw=0.8)

# Average V4 PSD
v4_psds = []
for ch in v4_idx:
    freqs, psd = compute_psd_welch(processed_lfp_monkey[:, ch], fs_processed)
    v4_psds.append(psd)
v4_avg = np.mean(v4_psds, axis=0)
v4_avg_smooth = gaussian_filter1d(v4_avg, sigma=2)
ax.loglog(freqs, v4_avg_smooth, color='darkred', lw=2.5, label='V4 Average')
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("PSD (a.u./Hz)")
ax.set_title("V4 LFP Power Spectrum")
ax.legend()
ax.grid(True, ls='--', alpha=0.3)

plt.tight_layout()
plt.show()

---
# Part 3: Preparing LFP for TVAR Modeling

Both pipelines converge here for time-varying autoregressive analysis.
Key preprocessing for AR/TVAR:
1. **Narrow-band filtering** to focus on frequency band of interest
2. **Z-score normalization** for stable AR coefficient estimation
3. **Downsampling** to keep AR order tractable

In [ ]:
# ============================================================
# Prepare LFP for TVAR Modeling
# ============================================================

def prepare_for_tvar(lfp: np.ndarray, fs: float, 
                     band: tuple = (1.0, 50.0),
                     target_fs: float = 250.0,
                     zscore: bool = True) -> tuple:
    """
    Prepare LFP signal for TVAR modeling.
    
    Parameters
    ----------
    lfp : np.ndarray
        Input LFP (1D)
    fs : float
        Sampling frequency (Hz)
    band : tuple
        (low_hz, high_hz) bandpass filter cutoffs
    target_fs : float
        Target sampling rate after downsampling
    zscore : bool
        Whether to z-score normalize
        
    Returns
    -------
    tuple
        (processed_signal, new_fs, time_vector)
    """
    from scipy.signal import butter, sosfiltfilt, decimate
    
    # Bandpass filter
    low, high = band
    sos = butter(4, [low / (fs/2), high / (fs/2)], btype='bandpass', output='sos')
    x_filt = sosfiltfilt(sos, lfp)
    
    # Downsample
    new_fs = fs
    if fs > target_fs:
        r = int(np.floor(fs / target_fs))
        x_filt = decimate(x_filt, r, ftype='iir', zero_phase=True)
        new_fs = fs / r
    
    # Z-score
    if zscore:
        x_filt = (x_filt - np.mean(x_filt)) / (np.std(x_filt) + 1e-12)
    
    time = np.arange(len(x_filt)) / new_fs
    
    return x_filt.astype(float), new_fs, time


# Prepare mouse LFP for TVAR (theta-gamma band)
xs_mouse, fs_tvar_mouse, time_tvar_mouse = prepare_for_tvar(
    raw_lfp_mouse, fs_mouse, band=(4.0, 100.0), target_fs=250.0
)

# Prepare monkey LFP (single channel) for TVAR (alpha-gamma band)
xs_monkey, fs_tvar_monkey, time_tvar_monkey = prepare_for_tvar(
    processed_lfp_monkey[:, 0], fs_processed, band=(1.0, 50.0), target_fs=250.0
)

print(f"Mouse TVAR-ready: {len(xs_mouse)} samples at {fs_tvar_mouse} Hz")
print(f"Monkey TVAR-ready: {len(xs_monkey)} samples at {fs_tvar_monkey} Hz")

In [ ]:
# ============================================================
# Visualization: TVAR-ready Signals
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(12, 6))

# Mouse: time-domain snippet
ax = axes[0, 0]
snippet_len = min(2000, len(xs_mouse))
ax.plot(time_tvar_mouse[:snippet_len], xs_mouse[:snippet_len], 'k', lw=0.8)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude (z-scored)")
ax.set_title("Mouse LFP (TVAR-ready)")
ax.grid(True, ls='--', alpha=0.3)

# Mouse: PSD
ax = axes[0, 1]
freqs_m, psd_m = compute_psd_welch(xs_mouse, fs_tvar_mouse, fmin=1, fmax=100)
ax.semilogy(freqs_m, gaussian_filter1d(psd_m, sigma=2), 'k', lw=2)
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("PSD")
ax.set_title("Mouse LFP PSD")
ax.grid(True, ls='--', alpha=0.3)

# Monkey: time-domain snippet
ax = axes[1, 0]
snippet_len = min(2000, len(xs_monkey))
ax.plot(time_tvar_monkey[:snippet_len], xs_monkey[:snippet_len], 'darkred', lw=0.8)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude (z-scored)")
ax.set_title("Monkey LFP (TVAR-ready)")
ax.grid(True, ls='--', alpha=0.3)

# Monkey: PSD
ax = axes[1, 1]
freqs_k, psd_k = compute_psd_welch(xs_monkey, fs_tvar_monkey, fmin=1, fmax=100)
ax.semilogy(freqs_k, gaussian_filter1d(psd_k, sigma=2), 'darkred', lw=2)
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("PSD")
ax.set_title("Monkey LFP PSD")
ax.grid(True, ls='--', alpha=0.3)

plt.tight_layout()
plt.show()

---
# Summary: Pipeline Comparison

| Aspect | Mouse (Hippocampus) | Monkey (Visual Cortex) |
|--------|---------------------|------------------------|
| **Sampling Rate** | ~20 kHz (high-density) | ~500 Hz (Utah arrays) |
| **Key Feature** | Sharp-wave ripples (80-140 Hz) | V1/V4 oscillations |
| **Preprocessing** | Bandpass + Hilbert envelope | Notch + Bandpass |
| **Event Detection** | Threshold-based ripple detection | PSD-based characterization |
| **Multi-channel** | Single electrode focus | Multi-array grouping |
| **Target Fs for TVAR** | 250-500 Hz | 250-500 Hz |

---

## Real Data Loading Templates

Use these templates when loading actual experimental data:

In [ ]:
# ============================================================
# Template: Load Real Mouse LFP Data
# ============================================================
# Uncomment and modify for your data:

# filepath = '/path/to/your/mouse_lfp_data.npy'
# channel_id = 0  # Select channel
# fs_mouse = 20000  # Typical for Neuropixels or high-density probes

# # Load
# raw_lfp_mouse = np.load(filepath)[channel_id]
# time_mouse = np.arange(len(raw_lfp_mouse)) / fs_mouse

# # Process for ripple detection
# ripple_band_lfp = bandpass_filter(raw_lfp_mouse, 80, 140, fs_mouse)
# analytic_signal = hilbert(ripple_band_lfp)
# envelope = np.abs(analytic_signal)
# ripple_intervals, threshold = detect_ripples(envelope, time_mouse)

# # Prepare for TVAR
# xs_mouse, fs_tvar_mouse, time_tvar = prepare_for_tvar(
#     raw_lfp_mouse, fs_mouse, band=(4.0, 100.0), target_fs=250.0
# )

print("Mouse data loading template ready.")

In [ ]:
# ============================================================
# Template: Load Real Monkey LFP Data
# ============================================================
# Uncomment and modify for your data:

# import scipy.io as sio
# import os

# data_dir = '/path/to/monkey/lfp/data'
# file_names = ['NSP1_array1_LFP.mat', 'NSP1_array2_LFP.mat', ...]

# # Load all arrays
# data = []
# for file_name in file_names:
#     full_path = os.path.join(data_dir, file_name)
#     data.append(sio.loadmat(full_path))

# # Concatenate
# fs_monkey = 500  # Typical for Utah arrays
# LFP = np.concatenate([d['lfp'] for d in data], axis=1)
# Array_ID = np.concatenate([d['Array_ID'] for d in data])

# # Separate V1/V4 by array ID (adjust based on your array mapping)
# V4_chan = np.where((Array_ID == 2) | (Array_ID == 3))[0]
# V1_chan = np.where((Array_ID != 2) & (Array_ID != 3))[0]

# # Process
# processed_lfp, fs_processed = preprocess_monkey_lfp(
#     LFP, fs_monkey, hp=0.5, lp=200.0, notch_freqs=(60.0, 120.0)
# )

# # Prepare single channel for TVAR
# xs_monkey, fs_tvar_monkey, time_tvar = prepare_for_tvar(
#     processed_lfp[:, 0], fs_processed, band=(1.0, 50.0), target_fs=250.0
# )

print("Monkey data loading template ready.")